#### Following [Google’s Neural Machine Translation System: Bridging the Gap  between Human and Machine Translation]( https://arxiv.org/pdf/1609.08144.pdf )

In [3]:

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
from utils.dataset import NMTDataset
from tensorflow.python.ops import math_ops

In [4]:
BUFFER_SIZE = 32000
BATCH_SIZE = 64
num_examples = 30000

dataset_creator = NMTDataset('en-spa')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

2646016/2638744 [==============================] - 0s 0us/step


In [5]:
print("Inpute Vocabulary Size: {}".format(len(inp_lang.word_index)))
print("Target Vocabulary Size: {}".format(len(targ_lang.word_index)))

Inpute Vocabulary Size: 9414
Target Vocabulary Size: 4935


In [7]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

In [8]:
# Define some useful parameters for further use

vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dim = 128
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE

## [Google NMT Paper's Architecture](https://arxiv.org/pdf/1609.08144.pdf)

We write the encoder-decoder modules as described in the original paper. 
<p align="center">
  <img src="https://slideplayer.com/slide/13133175/79/images/5/Model+Architecture+of+GNMT.jpg" />
</p>

* There are residual connection from *3rd* layer onwards in both Encoder and Decoders.
* The original paper uses a feedforward network for attention module. However, we'll use Bahdanau Attention. (In case you want to implement the feedforward attnetion module it's pretty straight forward.)
  * Pass *h<sub>t-1</sub>* and *enc_outputs* to FFN layer, and return context vector of length *(BATCH_SIZE, units)*



In [9]:
##### GNMT Encoder - 8 layers, first layer is bidirectional

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.num_layers = 8
    self.sub_layer = [None]*self.num_layers
    self.hidden_hs = [None]*self.num_layers
    self.hidden_cs = [None]*self.num_layers
    self.num_residual_layers = self.num_layers - 2
    self.num_bi_layers = 1
    self.num_ui_layers = self.num_layers - self.num_bi_layers
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    #---- One Bidirectional Layer -----##
    for i in range(self.num_bi_layers):
      self.sub_layer[i] = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.enc_units, return_sequences=True,                                                                    
                                                  return_state=True, recurrent_initializer='glorot_uniform'))
    
    self.bi_h = tf.keras.layers.Dense(self.enc_units)
    self.bi_c = tf.keras.layers.Dense(self.enc_units)
    
    ##----- Rest Unidirectional Layers ------##
    for i in range(self.num_bi_layers, self.num_layers):
      self.sub_layer[i] = tf.keras.layers.LSTM(self.enc_units, return_sequences=True, 
                                       return_state=True, recurrent_initializer='glorot_uniform')
    
    
    


  def call(self, x):
    x = self.embedding(x)
    
    for i in range(self.num_bi_layers):
      x, h_fw, c_fw, h_bw, c_bw = self.sub_layer[i](x)
    
    self.hidden_hs[i] = self.bi_h(tf.concat([h_fw, h_bw], axis=1))
    self.hidden_cs[i] = self.bi_c(tf.concat([c_fw, c_bw], axis=1))
    
    
    for i in range(self.num_bi_layers, self.num_layers):
      prev_x = x
      x, self.hidden_hs[i], self.hidden_cs[i] = self.sub_layer[i](x) 
      if(i !=1 ):
        x = prev_x + x  ## Residual Connection
        
    return x, self.hidden_hs, self.hidden_cs

In [10]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
# sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print(len(sample_h))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h[0].shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c[0].shape))

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
8
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [11]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    ######## query : Decoder's Last Hidden State  #####
    ######## values : Encoder Outputs             #####

    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hid den size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [12]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_h[0], sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 16, 1)


In [14]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.num_layers = 8
    self.sub_layer = [None]*self.num_layers

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    for i in range(self.num_layers):
      self.sub_layer[i] = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, 
                                       return_state=True, recurrent_initializer='glorot_uniform')
    
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden_hs, hidden_cs, enc_outputs):
    # enc_output shape == (batch_size, max_length, hidden_size)
    ## Attention calculated from last on query=last sub layer enc_h of encoder 
    context_vector, attention_weights = self.attention(hidden_hs[self.num_layers-1], enc_outputs)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    dec_hs = [None]*self.num_layers
    dec_cs = [None]*self.num_layers
    for i in range(self.num_layers):
      prev_x = x
      x, dec_hs[i] , dec_cs[i] = self.sub_layer[i](x, [hidden_hs[i], hidden_cs[i]])
      if(i > 1):
        x = prev_x + x   # Residual Connection
      
        

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, dec_hs, dec_cs, attention_weights

In [15]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, dec_hs, dec_cs, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_h, sample_c, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print('Decoder_h shape: ', dec_hs[0].shape)
print('Decoder_c shape: ', dec_cs[0].shape)

Decoder output shape: (batch_size, vocab size) (64, 4936)
Decoder_h shape:  (64, 1024)
Decoder_c shape:  (64, 1024)


## Define the optimizer and the loss function

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [18]:
@tf.function
def train_step(inp, targ):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp)
    # print("here")

    dec_hs = enc_h
    dec_cs = enc_c

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hs, dec_cs, _ = decoder(dec_input, dec_hs, dec_cs, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [29]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)

  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.5272
Epoch 1 Batch 100 Loss 0.5730
Epoch 1 Batch 200 Loss 0.6099
Epoch 1 Batch 300 Loss 0.6561
Epoch 1 Loss 0.4801
Time taken for 1 epoch 120.6132595539093 sec

Epoch 2 Batch 0 Loss 0.4649
Epoch 2 Batch 100 Loss 0.5117
Epoch 2 Batch 200 Loss 0.5340
Epoch 2 Batch 300 Loss 0.5911
Epoch 2 Loss 0.4289
Time taken for 1 epoch 136.02092504501343 sec

Epoch 3 Batch 0 Loss 0.4108
Epoch 3 Batch 100 Loss 0.4487
Epoch 3 Batch 200 Loss 0.4856
Epoch 3 Batch 300 Loss 0.5084
Epoch 3 Loss 0.3824
Time taken for 1 epoch 120.5916314125061 sec

Epoch 4 Batch 0 Loss 0.3674
Epoch 4 Batch 100 Loss 0.3970
Epoch 4 Batch 200 Loss 0.4439
Epoch 4 Batch 300 Loss 0.4721
Epoch 4 Loss 0.3388
Time taken for 1 epoch 136.07186245918274 sec

Epoch 5 Batch 0 Loss 0.3583
Epoch 5 Batch 100 Loss 0.3837
Epoch 5 Batch 200 Loss 0.3817
Epoch 5 Batch 300 Loss 0.3990
Epoch 5 Loss 0.3024
Time taken for 1 epoch 120.45960569381714 sec

Epoch 6 Batch 0 Loss 0.2923
Epoch 6 Batch 100 Loss 0.3105
Epoch 6 Batch 200 L

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [24]:
def evaluate(sentence):
  sentence = dataset_creator.preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_input,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  # hidden = [tf.zeros((1, units)), tf.zeros((1,units)), tf.zeros((1, units)), tf.zeros((1,units))]
  enc_out, enc_h, enc_c = encoder(inputs)

  dec_hs = enc_h
  dec_cs = enc_c

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_output):
    predictions, dec_hs, dec_cs, attention_weights = decoder(dec_input,
                                                         dec_hs, dec_cs,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [30]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [31]:
translate(u'hace mucho frio aqui.')

Input: <start> hace mucho frio aqui . <end>
Predicted translation: it s monday today . <end> 


In [32]:
translate(u'esta es mi vida.')

Input: <start> esta es mi vida . <end>
Predicted translation: this is my life . <end> 


In [33]:
translate(u'¿todavia estan en casa?')

Input: <start> ¿ todavia estan en casa ? <end>
Predicted translation: are you afraid of tom ? <end> 


In [34]:
# wrong translation
translate(u'trata de averiguarlo.')

Input: <start> trata de averiguarlo . <end>
Predicted translation: try to figure it out . <end> 


In [36]:
def translate_batch(test_dataset):
  with open('output_text.txt', 'w') as f:
    for (inputs, targets) in test_dataset:
      outputs = np.zeros((BATCH_SIZE, max_length_output),dtype=np.int16)
      # hidden_state = [tf.zeros((BATCH_SIZE, units)), tf.zeros((BATCH_SIZE, units)), tf.zeros((BATCH_SIZE, units)), tf.zeros((BATCH_SIZE, units))] 
      enc_output, dec_hs, dec_cs = encoder(inputs)
      dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
      for t in range(max_length_output):
        preds, dec_hs, dec_cs,  _ = decoder(dec_input, dec_hs, dec_cs, enc_output)
        predicted_id = tf.argmax(preds, axis=1).numpy()
        outputs[:, t] = predicted_id
        dec_input = tf.expand_dims(predicted_id, 1)
      outputs = targ_lang.sequences_to_texts(outputs)
      for t, item in enumerate(outputs):
        try:
          i = item.index('<end>')
          f.write("%s\n" %item[:i])
        except: 
          f.write("%s \n" % item)

outputs = translate_batch(val_dataset)

In [ ]:
!head output_text.txt
! wc -l output_text.txt

come see me . 
i like my family . 
they gave it . 
tom died the fence . 
listen . 
put it back . 
let me finish . 
tom is nervous . 
he fixed a diary . 
this one works . 
5952 output_text.txt
